# About this notebook:
- We will be using articles IDs to collect each articles' information, including:
    - Title
    - Abstract
    - Given Mesh Terms
    - Publication Date
- This notebook is part of a series of notebooks under 'Step 2 Articles Collection', where each notebook contain articles published in a particular year.

# Import Libraries and Articles' IDs

In [1]:
# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from Bio import Entrez
import xml.etree.ElementTree as ET

import time
from tqdm import tqdm

# Set the email address associated with your NCBI account
Entrez.email = "geok1723@gmail.com"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df=pd.read_csv("../Step_1_ID_Collection/IDs_2015.csv")

In [2]:
df.head()

,Date_str,ID,Date,Month,Year
0,2015/01/08,25565897,2015-01-08,1,2015
1,2015/01/27,25619629,2015-01-27,1,2015
2,2015/01/06,25557018,2015-01-06,1,2015
3,2015/01/09,25568442,2015-01-09,1,2015
4,2015/01/30,25631110,2015-01-30,1,2015


# Collecting Articles in groups of 100<br>
> The IDs are split into mini-dataframes containing 100 IDs for scraping. <br>
> Output will contain about 360 mini dataframes of articles that are published in this particular year. <br>
> This process was executed using Kaggle, and resulting dataframes are stored in the respective zip-folder. <br>

In [3]:
# determine the group number for each row
group_num = (df.index // 100) + 1

# map the group number to a group label (G1 to G6)
group_label = 'G' + group_num.astype(str)

# add the new column to the DataFrame
df['sub_group'] = group_label

# print(df)
# print(df['sub_group'].nunique())

df['Date']=pd.to_datetime(df['Date'],yearfirst=True)

# group the DataFrame by "Sub_group"
grouped = df.groupby('sub_group')

# create an empty dictionary to store the smaller DataFrames
df_dict = {}

# loop over the groups and create smaller DataFrames
for subgroup, subgroup_df in grouped:
    # construct the name of the smaller DataFrame
    year = subgroup_df['Year'].iloc[0]
    month = subgroup_df['Date'].iloc[0].strftime('%m')
    df_name = f"df_{year}_{month}_{subgroup}"
    # add the smaller DataFrame to the dictionary
    df_dict[df_name] = subgroup_df.copy()

df_ls = list(df_dict.keys())

df_ls.sort()

In [7]:
output_df = {}
for grp in tqdm(df_ls):
    _df = df_dict[grp]
    _s = _df[['ID','Date_str']] 

    print(f"Creating Dataframe of articles for {grp}")
    abstracts = []
    titles = []
    ids = []
    MeSH = []
    Pub_Date = []

    for index,row in tqdm(_s.iterrows()):	

        pub_date = row['Date_str']
        pmid = row['ID']


        try:
            handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="xml")

            xml_data = handle.read()
            handle.close()

            #obtain abstract and title
            root = ET.fromstring(xml_data)
            title = root.find(".//ArticleTitle").text.strip()
            abst = root.findall(".//AbstractText")

            abstract_text = ""
            for abstract in abst:
                abstract_text += abstract.text

            
            #obtain MeSH
            mesh_terms = ['']
            mesh_list = root.findall(".//MeshHeadingList/MeshHeading")
            for mesh in mesh_list:
                descriptor_name = mesh.find(".//DescriptorName").text
                qualifiers = mesh.findall(".//QualifierName")
                qualifier_names = [qualifier.text for qualifier in qualifiers]
                mesh_terms.append((descriptor_name, qualifier_names))
                
            MeSH.append(mesh_terms)            
            ids.append(pmid)
            abstracts.append(abstract_text)
            titles.append(title)
            Pub_Date.append(pub_date)

        except:
            print(f"Error here in ID{pmid}")

    # Store the data in a pandas dataframe
    data = {"ID": ids, "title": titles,"Pub_Date":Pub_Date, "abstract": abstracts, "MeSH_term":MeSH}
    data_df = pd.DataFrame(data)
    df_name = f"ABSTRACT{grp}"
    output_df[df_name] = data_df
    data_df.to_csv(f"{df_name}.csv", index=False)
    print(output_df.keys())

  0%|          | 0/360 [00:00<?, ?it/s]

Creating Dataframe of articles for df_2015_01_G1



0it [00:00, ?it/s]
1it [00:01,  1.51s/it]
2it [00:02,  1.27s/it]
3it [00:03,  1.21s/it]
4it [00:05,  1.24s/it]
5it [00:06,  1.27s/it]
6it [00:08,  1.54s/it]
7it [00:09,  1.49s/it]
8it [00:10,  1.38s/it]
9it [00:12,  1.35s/it]
10it [00:13,  1.33s/it]
11it [00:14,  1.26s/it]
12it [00:15,  1.21s/it]
13it [00:17,  1.31s/it]
14it [00:18,  1.33s/it]
15it [00:20,  1.37s/it]
16it [00:21,  1.28s/it]
17it [00:22,  1.24s/it]
18it [00:23,  1.29s/it]
19it [00:25,  1.33s/it]
20it [00:26,  1.27s/it]
21it [00:27,  1.27s/it]
22it [00:28,  1.21s/it]

Error here in ID28316700



23it [00:31,  1.70s/it]
24it [00:32,  1.54s/it]
25it [00:33,  1.46s/it]
26it [00:34,  1.35s/it]
27it [00:36,  1.26s/it]
28it [00:37,  1.28s/it]
29it [00:38,  1.22s/it]
30it [00:39,  1.19s/it]
31it [00:40,  1.16s/it]
32it [00:41,  1.21s/it]
33it [00:43,  1.22s/it]
34it [00:44,  1.21s/it]
35it [00:45,  1.24s/it]
36it [00:46,  1.23s/it]
37it [00:48,  1.20s/it]
38it [00:49,  1.17s/it]
39it [00:50,  1.21s/it]
40it [00:51,  1.24s/it]
41it [00:52,  1.21s/it]
42it [00:54,  1.25s/it]
43it [00:55,  1.26s/it]
44it [00:56,  1.20s/it]
45it [00:57,  1.17s/it]
46it [00:58,  1.15s/it]
47it [01:00,  1.20s/it]
48it [01:01,  1.26s/it]
49it [01:02,  1.24s/it]
50it [01:04,  1.31s/it]
51it [01:05,  1.31s/it]
52it [01:06,  1.25s/it]
53it [01:08,  1.32s/it]
54it [01:09,  1.25s/it]
55it [01:10,  1.21s/it]
56it [01:11,  1.24s/it]
57it [01:12,  1.25s/it]
58it [01:14,  1.27s/it]
59it [01:15,  1.22s/it]
60it [01:16,  1.18s/it]
61it [01:17,  1.21s/it]
62it [01:18,  1.22s/it]
63it [01:20,  1.25s/it]
64it [01:21,  1

...


82it [01:38,  1.14s/it]
83it [01:39,  1.14s/it]
84it [01:40,  1.13s/it]
85it [01:41,  1.19s/it]
86it [01:43,  1.22s/it]
87it [01:44,  1.18s/it]
88it [01:45,  1.27s/it]
89it [01:47,  1.30s/it]
90it [01:48,  1.35s/it]
91it [01:49,  1.35s/it]
92it [01:51,  1.34s/it]
93it [01:52,  1.36s/it]
94it [01:53,  1.35s/it]
95it [01:55,  1.36s/it]
96it [01:56,  1.31s/it]
97it [01:57,  1.31s/it]
98it [01:59,  1.31s/it]
99it [02:00,  1.41s/it]
100it [02:01,  1.22s/it]
 96%|█████████▌| 346/360 [11:55:43<28:54, 123.86s/it]

dict_keys(['ABSTRACTdf_2015_01_G1', 'ABSTRACTdf_2015_01_G10', 'ABSTRACTdf_2015_01_G11', 'ABSTRACTdf_2015_01_G12', 'ABSTRACTdf_2015_01_G13', 'ABSTRACTdf_2015_01_G14', 'ABSTRACTdf_2015_01_G15', 'ABSTRACTdf_2015_01_G16', 'ABSTRACTdf_2015_01_G17', 'ABSTRACTdf_2015_01_G18', 'ABSTRACTdf_2015_01_G19', 'ABSTRACTdf_2015_01_G2', 'ABSTRACTdf_2015_01_G20', 'ABSTRACTdf_2015_01_G21', 'ABSTRACTdf_2015_01_G22', 'ABSTRACTdf_2015_01_G23', 'ABSTRACTdf_2015_01_G24', 'ABSTRACTdf_2015_01_G25', 'ABSTRACTdf_2015_01_G26', 'ABSTRACTdf_2015_01_G27', 'ABSTRACTdf_2015_01_G28', 'ABSTRACTdf_2015_01_G29', 'ABSTRACTdf_2015_01_G3', 'ABSTRACTdf_2015_01_G30', 'ABSTRACTdf_2015_01_G4', 'ABSTRACTdf_2015_01_G5', 'ABSTRACTdf_2015_01_G6', 'ABSTRACTdf_2015_01_G7', 'ABSTRACTdf_2015_01_G8', 'ABSTRACTdf_2015_01_G9', 'ABSTRACTdf_2015_02_G31', 'ABSTRACTdf_2015_02_G32', 'ABSTRACTdf_2015_02_G33', 'ABSTRACTdf_2015_02_G34', 'ABSTRACTdf_2015_02_G35', 'ABSTRACTdf_2015_02_G36', 'ABSTRACTdf_2015_02_G37', 'ABSTRACTdf_2015_02_G38', 'ABSTRACTd


0it [00:00, ?it/s]
1it [00:01,  1.10s/it]
2it [00:02,  1.20s/it]
3it [00:03,  1.18s/it]
4it [00:04,  1.25s/it]
5it [00:06,  1.22s/it]
6it [00:07,  1.26s/it]
7it [00:08,  1.21s/it]
8it [00:09,  1.18s/it]
9it [00:10,  1.18s/it]
10it [00:11,  1.15s/it]
11it [00:13,  1.21s/it]
12it [00:14,  1.21s/it]
13it [00:15,  1.25s/it]
14it [00:16,  1.20s/it]
15it [00:17,  1.17s/it]
16it [00:19,  1.17s/it]
17it [00:20,  1.21s/it]
18it [00:21,  1.21s/it]
19it [00:22,  1.17s/it]
20it [00:24,  1.22s/it]
21it [00:25,  1.20s/it]
22it [00:26,  1.17s/it]
23it [00:27,  1.22s/it]
24it [00:28,  1.22s/it]
25it [00:30,  1.20s/it]
26it [00:31,  1.20s/it]
27it [00:32,  1.18s/it]
28it [00:33,  1.16s/it]
29it [00:34,  1.24s/it]
30it [00:35,  1.20s/it]
31it [00:37,  1.18s/it]
32it [00:38,  1.21s/it]
33it [00:39,  1.19s/it]
34it [00:40,  1.16s/it]
35it [00:41,  1.20s/it]
36it [00:43,  1.24s/it]
37it [00:44,  1.20s/it]
38it [00:45,  1.17s/it]
39it [00:46,  1.20s/it]
40it [00:48,  1.31s/it]
41it [00:49,  1.30s/it]
42it 

dict_keys(['ABSTRACTdf_2015_01_G1', 'ABSTRACTdf_2015_01_G10', 'ABSTRACTdf_2015_01_G11', 'ABSTRACTdf_2015_01_G12', 'ABSTRACTdf_2015_01_G13', 'ABSTRACTdf_2015_01_G14', 'ABSTRACTdf_2015_01_G15', 'ABSTRACTdf_2015_01_G16', 'ABSTRACTdf_2015_01_G17', 'ABSTRACTdf_2015_01_G18', 'ABSTRACTdf_2015_01_G19', 'ABSTRACTdf_2015_01_G2', 'ABSTRACTdf_2015_01_G20', 'ABSTRACTdf_2015_01_G21', 'ABSTRACTdf_2015_01_G22', 'ABSTRACTdf_2015_01_G23', 'ABSTRACTdf_2015_01_G24', 'ABSTRACTdf_2015_01_G25', 'ABSTRACTdf_2015_01_G26', 'ABSTRACTdf_2015_01_G27', 'ABSTRACTdf_2015_01_G28', 'ABSTRACTdf_2015_01_G29', 'ABSTRACTdf_2015_01_G3', 'ABSTRACTdf_2015_01_G30', 'ABSTRACTdf_2015_01_G4', 'ABSTRACTdf_2015_01_G5', 'ABSTRACTdf_2015_01_G6', 'ABSTRACTdf_2015_01_G7', 'ABSTRACTdf_2015_01_G8', 'ABSTRACTdf_2015_01_G9', 'ABSTRACTdf_2015_02_G31', 'ABSTRACTdf_2015_02_G32', 'ABSTRACTdf_2015_02_G33', 'ABSTRACTdf_2015_02_G34', 'ABSTRACTdf_2015_02_G35', 'ABSTRACTdf_2015_02_G36', 'ABSTRACTdf_2015_02_G37', 'ABSTRACTdf_2015_02_G38', 'ABSTRACTd


0it [00:00, ?it/s]
1it [00:01,  1.11s/it]
2it [00:02,  1.26s/it]
3it [00:04,  1.40s/it]
4it [00:05,  1.37s/it]
5it [00:06,  1.33s/it]
6it [00:07,  1.25s/it]
7it [00:08,  1.19s/it]
8it [00:09,  1.18s/it]
9it [00:11,  1.24s/it]
10it [00:12,  1.25s/it]
11it [00:13,  1.21s/it]
12it [00:14,  1.19s/it]
13it [00:15,  1.15s/it]
14it [00:17,  1.20s/it]
15it [00:18,  1.23s/it]
16it [00:20,  1.40s/it]
17it [00:21,  1.32s/it]
18it [00:22,  1.33s/it]
19it [00:23,  1.25s/it]
20it [00:25,  1.26s/it]
21it [00:26,  1.27s/it]
22it [00:27,  1.28s/it]
23it [00:28,  1.23s/it]
24it [00:29,  1.19s/it]
25it [00:31,  1.23s/it]
26it [00:32,  1.31s/it]
27it [00:34,  1.30s/it]
28it [00:35,  1.30s/it]
29it [00:36,  1.25s/it]
30it [00:37,  1.21s/it]
31it [00:38,  1.23s/it]
32it [00:40,  1.26s/it]
33it [00:41,  1.25s/it]
34it [00:42,  1.20s/it]
35it [00:43,  1.25s/it]
36it [00:45,  1.26s/it]
37it [00:46,  1.21s/it]
38it [00:47,  1.17s/it]
39it [00:48,  1.15s/it]
40it [00:50,  1.41s/it]
41it [00:51,  1.32s/it]
42it 